In [10]:
import tweepy

In [23]:
with open('keys','r') as key_file:
    key = key_file.readline().strip()
    secret = key_file.readline().strip()
    bearer = key_file.readline().strip()

client = tweepy.Client(bearer)

In [27]:
query = 'paredes de coura'

count = 10

s = client.search_recent_tweets(query=query)

In [29]:
s

Response(data=[<Tweet id=1584514583636500480 text='se calhar deixar a pulseira do paredes de coura no pulso é estúpido porque cozinhei e reparei agora cheira a fritos hmm que bom'>, <Tweet id=1584307506738008064 text='@vascossilva2 Eu com o Paredes de coura 😭😭'>, <Tweet id=1584240550617964544 text='@martaa_scm Vai ter mais sorte no paredes de coura'>, <Tweet id=1583826889650896896 text='RT @OhmSweetOhm: Convidem lá The Ginskeys para irem tocar ao Paredes De Coura de 2023  👉👈'>, <Tweet id=1583824851344961537 text='Convidem lá The Ginskeys para irem tocar ao Paredes De Coura de 2023  👉👈'>, <Tweet id=1583776503267209217 text='RT @tudosedesmonta: paredes de coura 2023 assim será'>, <Tweet id=1583515472871718912 text="RT @pedro_noodles: 1 minuto de paredes de coura '22💜 https://t.co/7J1EhDeuIk">, <Tweet id=1583515371415293952 text="RT @pedro_noodles: 1 minuto de paredes de coura '22💜 https://t.co/7J1EhDeuIk">, <Tweet id=1583471047558692864 text='@ParedesCoura  recebe OGRE electric de Maria 

In [30]:
for t in s.data:
    print(t)

se calhar deixar a pulseira do paredes de coura no pulso é estúpido porque cozinhei e reparei agora cheira a fritos hmm que bom
@vascossilva2 Eu com o Paredes de coura 😭😭
@martaa_scm Vai ter mais sorte no paredes de coura
RT @OhmSweetOhm: Convidem lá The Ginskeys para irem tocar ao Paredes De Coura de 2023  👉👈
Convidem lá The Ginskeys para irem tocar ao Paredes De Coura de 2023  👉👈
RT @tudosedesmonta: paredes de coura 2023 assim será
RT @pedro_noodles: 1 minuto de paredes de coura '22💜 https://t.co/7J1EhDeuIk
RT @pedro_noodles: 1 minuto de paredes de coura '22💜 https://t.co/7J1EhDeuIk
@ParedesCoura  recebe OGRE electric de Maria João ⬇️
https://t.co/Lm79PZHwBC 
#paredesdecoura #cultura #lercinema #musica #OGRE #mariajoao
paredes de Coura rendeu duas multa 260€, mto que bem


##### 